# 2D Transient Navier-Stokes Flow on Open/Closed Channel

<b>Problem to Solve</b> 

<b>Goals</b>

<b>Questions</b>

<b>Remarks</b>
1. pressure boundary conditions needs be modified to take closed boundary into account;
2. function ferrite_limiter!(): in order to obtain correct higher order convergence for time-dependent Dirichlet conditions, one needs to ensure that all the internal buffers are also correctly set at all times. The limiter helps guarantees this (as the time integrators do also other stuff than just calling your right-hand side). See e.g. [rosenbrock_perform_step.jl](https://github.com/SciML/OrdinaryDiffEq.jl/blob/7a4a5fb6d95f78e973b3d7bb41861058d5b00881/lib/OrdinaryDiffEqRosenbrock/src/rosenbrock_perform_step.jl#L1386-L1388)
3. Rodas5P() failed: results in MethodError: no method matching Rodas5P(; autodiff::Bool, step_limiter!::typeof(ferrite_limiter!))

In [1]:
#?Rodas5P()

### Unknown Stiffness Problems

When the stiffness of the problem is unknown, it is recommended you use a stiffness detection and auto-switching algorithm. These methods are multi-paradigm and allow for efficient solution of both stiff and non-stiff problems. The cost for auto-switching is very minimal, but the choices are restrained. They are a good go-to method when applicable.

For default tolerances, AutoTsit5(Rosenbrock23()) is a good choice. For lower tolerances, using AutoVern7 or AutoVern9 with Rodas4, KenCarp4, or Rodas5P can all be good choices depending on the problem. For very large systems (>1000 ODEs?), consider using lsoda.

##  Import Packages

In [3]:
using BlockArrays
using LinearAlgebra
using UnPack
using LinearSolve 
using SparseArrays
using Ferrite
using FerriteGmsh 
using OrdinaryDiffEq
using DifferentialEquations
using Plots 
using WriteVTK

## Section 1: Introduction 

More later.

## Section 2: Definition of Structs 

In [4]:
#..pass data to RHS of time integration 
struct RHSparams
    K::SparseMatrixCSC
    ch::ConstraintHandler
    dh::DofHandler
    cellvalues_v::CellValues
    u::Vector
end

struct FreeDofErrorNorm
    ch::ConstraintHandler
end

## Section 3: Generate 2D Mesh

In [5]:
nelem = 10
H = 0.25; L = 4*H 
nels  = (4*nelem, nelem) # number of elements in each spatial direction
left  = Vec((0., 0.))    # start point for geometry 
right = Vec((L, H,)) # end point for geometry
grid = generate_grid(Quadrilateral,nels,left,right);

In [6]:
addvertexset!(grid, "corner", (x) -> x[1] ≈ 0.0 && x[2] ≈ 0.0)

Grid{2, Quadrilateral, Float64} with 400 Quadrilateral cells and 451 nodes

## Section 4: Set-up 

In [7]:
#?Dirichlet

In [8]:
#?update!

In [9]:
dim = 2 

ip_v = Lagrange{RefQuadrilateral, 2}()^dim
qr = QuadratureRule{RefQuadrilateral}(4)
cellvalues_v = CellValues(qr, ip_v);

ip_p = Lagrange{RefQuadrilateral, 1}()
cellvalues_p = CellValues(qr, ip_p);

dh = DofHandler(grid)
add!(dh, :v, ip_v)
add!(dh, :p, ip_p)
close!(dh);

ch = ConstraintHandler(dh);

#..wall boundary conditions.. 
nosplip_facet_names = ["top", "bottom"];
∂Ω_noslip = union(getfacetset.((grid,), nosplip_facet_names)...);
noslip_bc = Dirichlet(:v, ∂Ω_noslip, (x, t) -> Vec((0.0, 0.0)), [1, 2])
add!(ch, noslip_bc);

#..inflow boundary conditions..
∂Ω_inflow = getfacetset(grid, "left");
# vᵢₙ(t) = min(t * 1.5, 1.5) # ramped inflow velocity
vᵢₙ(t) = 1.5 
parabolic_inflow_profile(x,t) = Vec((1,0)) # Vec((4*vᵢₙ(t)*x[2]*(0.25-x[2]), 0.0))
inflow_bc = Dirichlet(:v, ∂Ω_inflow, parabolic_inflow_profile, [1, 2])
add!(ch, inflow_bc);

#..outflow boundary conditions: do nothing 
∂Ω_free = getfacetset(grid, "right");

close!(ch)
update!(ch, 0.0);

## Section 5: Functions for Mass and Stiffness Matrix Assembly and Other Functions   

In [10]:
function assemble_mass_matrix(cellvalues_v::CellValues, cellvalues_p::CellValues, M::SparseMatrixCSC, dh::DofHandler)
    # Allocate a buffer for the local matrix and some helpers, together with the assembler.
    n_basefuncs_v = getnbasefunctions(cellvalues_v)
    n_basefuncs_p = getnbasefunctions(cellvalues_p)
    n_basefuncs = n_basefuncs_v + n_basefuncs_p
    v▄, p▄ = 1, 2
    Mₑ = BlockedArray(zeros(n_basefuncs, n_basefuncs), [n_basefuncs_v, n_basefuncs_p], [n_basefuncs_v, n_basefuncs_p])

    # It follows the assembly loop as explained in the basic tutorials.
    mass_assembler = start_assemble(M)
    for cell in CellIterator(dh)
        fill!(Mₑ, 0)
        Ferrite.reinit!(cellvalues_v, cell)

        for q_point in 1:getnquadpoints(cellvalues_v)
            dΩ = getdetJdV(cellvalues_v, q_point)
            # Remember that we assemble a vector mass term, hence the dot product.
            # There is only one time derivative on the left hand side, so only one mass block is non-zero.
            for i in 1:n_basefuncs_v
                φᵢ = shape_value(cellvalues_v, q_point, i)
                for j in 1:n_basefuncs_v
                    φⱼ = shape_value(cellvalues_v, q_point, j)
                    Mₑ[BlockIndex((v▄, v▄), (i, j))] += φᵢ ⋅ φⱼ * dΩ
                end
            end
        end
        assemble!(mass_assembler, celldofs(cell), Mₑ)
    end

    return M
end;

function assemble_stokes_matrix(cellvalues_v::CellValues, cellvalues_p::CellValues, ν, K::SparseMatrixCSC, dh::DofHandler)
    # Again, some buffers and helpers
    n_basefuncs_v = getnbasefunctions(cellvalues_v)
    n_basefuncs_p = getnbasefunctions(cellvalues_p)
    n_basefuncs = n_basefuncs_v + n_basefuncs_p
    v▄, p▄ = 1, 2
    Kₑ = BlockedArray(zeros(n_basefuncs, n_basefuncs), [n_basefuncs_v, n_basefuncs_p], [n_basefuncs_v, n_basefuncs_p])

    # Assembly loop
    stiffness_assembler = start_assemble(K)
    for cell in CellIterator(dh)
        # Don't forget to initialize everything
        fill!(Kₑ, 0)

        Ferrite.reinit!(cellvalues_v, cell)
        Ferrite.reinit!(cellvalues_p, cell)

        for q_point in 1:getnquadpoints(cellvalues_v)
            dΩ = getdetJdV(cellvalues_v, q_point)

            for i in 1:n_basefuncs_v
                ∇φᵢ = shape_gradient(cellvalues_v, q_point, i)
                for j in 1:n_basefuncs_v
                    ∇φⱼ = shape_gradient(cellvalues_v, q_point, j)
                    Kₑ[BlockIndex((v▄, v▄), (i, j))] -= ν * ∇φᵢ ⊡ ∇φⱼ * dΩ
                end
            end

            for j in 1:n_basefuncs_p
                ψ = shape_value(cellvalues_p, q_point, j)
                for i in 1:n_basefuncs_v
                    divφ = shape_divergence(cellvalues_v, q_point, i)
                    Kₑ[BlockIndex((v▄, p▄), (i, j))] += (divφ * ψ) * dΩ
                    Kₑ[BlockIndex((p▄, v▄), (j, i))] += (ψ * divφ) * dΩ
                end
            end
        end

        # Assemble `Kₑ` into the Stokes matrix `K`.
        assemble!(stiffness_assembler, celldofs(cell), Kₑ)
    end
    return K
end

function ferrite_limiter!(u, _, p, t)
    update!(p.ch, t)
    return apply!(u, p.ch)
end

function navierstokes_rhs_element!(dvₑ, vₑ, cellvalues_v)
    n_basefuncs = getnbasefunctions(cellvalues_v)
    for q_point in 1:getnquadpoints(cellvalues_v)
        dΩ = getdetJdV(cellvalues_v, q_point)
        ∇v = function_gradient(cellvalues_v, q_point, vₑ)
        v = function_value(cellvalues_v, q_point, vₑ)
        for j in 1:n_basefuncs
            φⱼ = shape_value(cellvalues_v, q_point, j)

            dvₑ[j] -= v ⋅ ∇v' ⋅ φⱼ * dΩ
        end
    end
    return
end

function navierstokes!(du, u_uc, p::RHSparams, t)

    @unpack K, ch, dh, cellvalues_v, u = p

    u .= u_uc
    update!(ch, t)
    apply!(u, ch)

    # Linear contribution (Stokes operator)
    mul!(du, K, u) # du .= K * u

    # nonlinear contribution
    v_range = dof_range(dh, :v)
    n_basefuncs = getnbasefunctions(cellvalues_v)
    vₑ = zeros(n_basefuncs)
    duₑ = zeros(n_basefuncs)
    for cell in CellIterator(dh)
        Ferrite.reinit!(cellvalues_v, cell)
        v_celldofs = @view celldofs(cell)[v_range]
        vₑ .= @views u[v_celldofs]
        fill!(duₑ, 0.0)
        navierstokes_rhs_element!(duₑ, vₑ, cellvalues_v)
        assemble!(du, v_celldofs, duₑ)
    end
    return
end;

function navierstokes_jac_element!(Jₑ, vₑ, cellvalues_v)
    n_basefuncs = getnbasefunctions(cellvalues_v)
    for q_point in 1:getnquadpoints(cellvalues_v)
        dΩ = getdetJdV(cellvalues_v, q_point)
        ∇v = function_gradient(cellvalues_v, q_point, vₑ)
        v = function_value(cellvalues_v, q_point, vₑ)
        for j in 1:n_basefuncs
            φⱼ = shape_value(cellvalues_v, q_point, j)

            for i in 1:n_basefuncs
                φᵢ = shape_value(cellvalues_v, q_point, i)
                ∇φᵢ = shape_gradient(cellvalues_v, q_point, i)
                Jₑ[j, i] -= (φᵢ ⋅ ∇v' + v ⋅ ∇φᵢ') ⋅ φⱼ * dΩ
            end
        end
    end
    return
end

function navierstokes_jac!(J, u_uc, p, t)

    @unpack K, ch, dh, cellvalues_v, u = p

    u .= u_uc
    update!(ch, t)
    apply!(u, ch)

    # Linear contribution (Stokes operator)
    # Here we assume that J has exactly the same structure as K by construction
    nonzeros(J) .= nonzeros(K)

    assembler = start_assemble(J; fillzero = false)

    # Assemble variation of the nonlinear term
    n_basefuncs = getnbasefunctions(cellvalues_v)
    Jₑ = zeros(n_basefuncs, n_basefuncs)
    vₑ = zeros(n_basefuncs)
    v_range = dof_range(dh, :v)
    for cell in CellIterator(dh)
        Ferrite.reinit!(cellvalues_v, cell)
        v_celldofs = @view celldofs(cell)[v_range]

        vₑ .= @views u[v_celldofs]
        fill!(Jₑ, 0.0)
        navierstokes_jac_element!(Jₑ, vₑ, cellvalues_v)
        assemble!(assembler, v_celldofs, Jₑ)
    end

    return apply!(J, ch)
end

navierstokes_jac! (generic function with 1 method)

## Section 6: Define Initial Conditions  

In [11]:
K = allocate_matrix(dh);
viscosity = 1e3
K = assemble_stokes_matrix(cellvalues_v, cellvalues_p, viscosity, K, dh);

f = zeros(ndofs(dh))
update!(ch, 0.)
apply!(K, f, ch)
uinit = K \ f;

VTKGridFile("init-channel-2d", dh) do vtk
    write_solution(vtk, dh, uinit)
    Ferrite.write_constraints(vtk, ch)
end

VTKGridFile for the closed file "init-channel-2d.vtu".

## Section 7: Perform Naive Time Integration 

In [12]:
T = 6.0
Δt₀ = 0.001
Δt_save = 0.1

M = allocate_matrix(dh);
M = assemble_mass_matrix(cellvalues_v, cellvalues_p, M, dh);

K = allocate_matrix(dh);
viscosity = 1e3
K = assemble_stokes_matrix(cellvalues_v, cellvalues_p, viscosity, K, dh);

jac_sparsity = sparse(K);

apply!(M, ch)

p = RHSparams(K, ch, dh, cellvalues_v, copy(uinit))

rhs = ODEFunction(navierstokes!, mass_matrix = M; jac = navierstokes_jac!, jac_prototype = jac_sparsity)
problem = ODEProblem(rhs, uinit, (0.0, T), p);

(fe_norm::FreeDofErrorNorm)(u::Union{AbstractFloat, Complex}, t) = DiffEqBase.ODE_DEFAULT_NORM(u, t)
(fe_norm::FreeDofErrorNorm)(u::AbstractArray, t) = DiffEqBase.ODE_DEFAULT_NORM(u[fe_norm.ch.free_dofs], t)

problem = ODEProblem(navierstokes!, uinit, (0.0,0.1), p);
problem = ODEProblem(navierstokes!, uinit, (0.0,6.), p)
# sol = solve(problem, ImplicitEuler())
# sol = solve(problem, ImplicitEuler(autodiff=false)); 

ODEProblem with uType Vector{Float64} and tType Float64. In-place: true
timespan: (0.0, 6.0)
u0: 3853-element Vector{Float64}:
    1.0
    0.0
    0.0
    0.0
    1.0218754940492467
    0.2963794342268647
    1.0
    0.0
    0.0
    0.0
    0.820365789398192
    0.105650285955465
    0.998550999222628
    ⋮
    0.28499999999979775
    4.4162532418469294e-14
 4800.000000012827
    0.0
    0.0
    0.2849999999999995
    4.074151341209964e-14
    0.0
    0.0
    0.2849999999999345
    3.667589848814884e-14
    7.019931370505487e-9

In [13]:
p1 = plot(sol.t)
p2 = bar(sol.t,sol.t[2:end]-sol.t[1:end-1])
plot(p1,p2,layout=(1,2))

LoadError: UndefVarError: `sol` not defined

## Section 8: Perform Advanced Time Integration and Post-Processing 

We removed the option <i>d_discontinuities</i>.

We wish to store $u$ into a vector of vectors. 

In [45]:
# timestepper = Rodas5P(autodiff = false, step_limiter! = ferrite_limiter!);
# timestepper = AutoTsit5(Rosenbrock23())(autodiff = false, step_limiter! = ferrite_limiter!);
# timestepper = AutoTsit5(autodiff = false, step_limiter! = ferrite_limiter!);

#integrator = init(
#    problem, timestepper; initializealg = NoInit(), dt = Δt₀,
#    adaptive = true, abstol = 1.0e-4, reltol = 1.0e-5,
#    progress = true, progress_steps = 1,
#    verbose = true, internalnorm = FreeDofErrorNorm(ch), d_discontinuities = [1.0]
#);

timestepper = ImplicitEuler(autodiff=false)

integrator = init(
    problem, timestepper; dt = Δt₀,
    adaptive = true, abstol = 1.0e-4, reltol = 1.0e-5,
    progress = true, progress_steps = 1,
    verbose = true, internalnorm = FreeDofErrorNorm(ch)
);

pvd = paraview_collection("channel-2d")
# uhist = Vector{Vector{Float64}(undef, ndofs(dh))}
for (step, (u, t)) in enumerate(intervals(integrator))
    display(t)
    VTKGridFile("channel-2d-$step", dh) do vtk
        write_solution(vtk, dh, u)
        pvd[t] = vtk
    end
    uhist[step] = u 
end
vtk_save(pvd);

LoadError: TypeError: in Type, in parameter, expected Type, got a value of type Vector{Float64}

In [55]:
uhist = Vector{Vector{Float64}}[]
typeof(uhist[1])

LoadError: BoundsError: attempt to access 0-element Vector{Vector{Vector{Float64}}} at index [1]